# Comparando modelos de detección de anomalías

En este ejemplo vamos a diseñar un experimento para comparar todos los modelos basados en vecinos, basados en agrupamiento y HBOs que se han visto en este curso para resolver tres problemas de detección de anomalías del mundo real usando la librería PyOD.

Vamos a usar datasets de la ODDs (Outlier Detection DataSets (ODDS)) http://odds.cs.stonybrook.edu. Que van a ser: arrythmia, cardio y glass, que se encuentran comentados en el documento que tenéis disponible.
 

Las medidas que se van a analizar son sensibilidad, especificación, precisión y curva ROC. Para su análisis se va a realizar una división del conjunto de datos, en datos de train y datos de test (como se han visto en todos los ejemplos estudiados) con la proporción (70% para training y 30% para test). Además, como son métodos estocásticos se van a realizar 10 ejecuciones diferentes en cada uno de ellos.

Los algoritmos que se encuentran en la librería PyOD son:

        LOF: Local Outlier Factor
        COF: Connectiviy Outlier Factor
        CBLOF: Clustering-Based Local Outlier Factor
        kNN: k Nearest Neighbors (use the distance to the kth nearest neighbor as the outlier score)
        HBOS: Histogram-based Outlier Score




Paso 1. Instalar la librería PyOD

In [ ]:
!pip install PyOD

Paso 2. Importamos las librerías que necesitamos

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat


from pyod.models.cblof import CBLOF
from pyod.models.hbos import HBOS
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.cof import COF


from pyod.utils.utility import standardizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

import os
import sys
from time import time

Paso 3. Definimos la lista de datos que vamos a usar (3 en nuestro caso), previamente debemos haberlos cargado en google colaboration (como se ha explicado en el video), la semilla del generador de números aleatorios que se usará en los casos que sea necesario y las variables donde almacenaremos las métricas que se van a obtener para cada ejecución por cada algoritmo, se incluye el tiempo de cómputo.

In [3]:
# Definimos la lista de datos que vamos a usar 
mat_datasets = ['arrhythmia.mat',
                 'cardio.mat',
                 'glass.mat']


# Definimos las variables que se van a utilizar para almacenar los resultados de 
# cada conjunto de datos, todas tendrán las mismas columnas y se hará una
# tabla para comparar cada métrica de las estudiadas
columnas_tabla = ['Datos', '#Ejemplos', '#Dimensiones', 'Anomalías(%)',
              'HBOS', 'CBLOF', 'KNN', 'LOF', 'COF']


#Configuramos los parámetros de cada algoritmo para cada dataset
k_lof = [80,250,5]
k_cof = [5,250,5]
k_knn = [10,40,5]
cluster_cblof = [6,5,50]
bins_hbos = [13,3,13]

#Definimos las variables
se_tabla = pd.DataFrame(columns=columnas_tabla)
sp_tabla = pd.DataFrame(columns=columnas_tabla)
p_tabla = pd.DataFrame(columns=columnas_tabla)
roc_tabla = pd.DataFrame(columns=columnas_tabla)
time_tabla = pd.DataFrame(columns=columnas_tabla)

Paso 4. Vamos a ejecutar cada uno de los algoritmos con cada conjunto de datos, primero preparamos los datos normalizándolos y dividiéndolos en train y test según la proporción (70% y 30%), a continuación aplicamos los algoritmos con sus parámetros optimizados y almacenamos en las tablas los resultados obtenidos para cada una de las métricas consideradas.

In [ ]:
# Definimos la semilla que se va a utilizar, siendo un número aleatorio entre 1 y 50
random_state = np.random.RandomState(42)

#usamos los parámetros ajustados
parameter = -1

# Para cada dataset considerado en el paso anterior
for mat_dataset in mat_datasets:

    #Utilizamos los parámetros del dataset
    parameter = parameter + 1
    
    # Cargamos el conjunto de datos
    dataset = loadmat(os.path.join('sample_data', mat_dataset))

    # Separamos los atributos de la clase
    X = dataset['X']
    y = dataset['y'].ravel()

    # Obtenemos la fracción de anomalías que tiene el conjunto de datos
    # es una variable que necesitan los algoritmos
    anomalias_fraccion = np.count_nonzero(y) / len(y)
    anomalias_porcentaje = round(anomalias_fraccion * 100, ndigits=4)

    # Definimos las listas para almacenar los resultados
    se_list = [mat_dataset[:-4], X.shape[0], X.shape[1], anomalias_porcentaje]
    sp_list =[mat_dataset[:-4], X.shape[0], X.shape[1], anomalias_porcentaje]
    p_list = [mat_dataset[:-4], X.shape[0], X.shape[1], anomalias_porcentaje]
    roc_list = [mat_dataset[:-4], X.shape[0], X.shape[1], anomalias_porcentaje]
    time_list = [mat_dataset[:-4], X.shape[0], X.shape[1], anomalias_porcentaje]

    # Divimos los datos en train y test (70%, 30% respectivamente). También
    # se pueden usar otras formas de cross-validation 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                        random_state=random_state)

    # Normalizamos los datos (usamos el métodos de PyOD)
    X_train_norm, X_test_norm = standardizer(X_train, X_test)


    # Configuramos los algoritmos que vamos a usar
    classificadores = {'Cluster-based Local Outlier Factor': CBLOF(
            contamination=anomalias_fraccion, check_estimator=False,
            random_state=random_state, n_clusters = cluster_cblof[parameter]),
        'Histogram-base Outlier Detection (HBOS)': HBOS(
            contamination=anomalias_fraccion, n_bins=bins_hbos[parameter]),
        'K Nearest Neighbors (KNN)': KNN(contamination=anomalias_fraccion,
            n_neighbors = k_knn[parameter]),
        'Local Outlier Factor (LOF)': LOF(
            contamination=anomalias_fraccion, n_neighbors = k_lof[parameter]),
        'Connection Outlier Factor (COF)': COF(
            contamination=anomalias_fraccion, n_neighbors = k_cof[parameter]),
      }

    # Para cada clasificador definido
    for clf_name, clf in classificadores.items():
        # Inicializamos el tiempo
        t0 = time()
        # Entrenamos el modelos con los datos normalizados
        clf.fit(X_train_norm)

        # Obtenemos los resultados para los datos de train y test
        y_train_pred = clf.labels_ 
        y_test_pred = clf.predict(X_test_norm)  

       # test_scores = clf.decision_function(X_test_norm)
        # Una vez tenemos los resultados finalizamos el tiempo
        t1 = time()
        duracion = round(t1 - t0, ndigits=4)
        time_list.append(duracion)


        #Calculamos sensibilidad, especificidad y precision desde la matriz de confusión
        # para los datos de  test
        cm = confusion_matrix(y_test,y_test_pred)
        total=sum(sum(cm))
        sensitivity = cm[1,1]/(cm[1,0]+cm[1,1])
        specificity = cm[0,0]/(cm[0,0]+cm[0,1])
        precision = cm[1,1]/(cm[1,1]+cm[0,1])

        se = round(sensitivity, ndigits=4)
        sp = round(specificity, ndigits=4)
        p = round(precision, ndigits=4)
        roc = round(roc_auc_score(y_test, y_test_pred), ndigits=4)
    
        print('{clf_name} Se:{se}, Sp:{sp}, precisión:{p}, ROC:{roc}, '
              'tiempo ejecución: {duracion}'.format(clf_name=clf_name, se=se, sp=sp, roc=roc, p=p, duracion=duracion))

        se_list.append(se)
        sp_list.append(sp)
        p_list.append(p)
        roc_list.append(roc)


    # Una vez finalizadas las ejecuciones de todos los métodos,
    # pasamos la información a tabla
    temp_tabla = pd.DataFrame(time_list).transpose()
    temp_tabla.columns = columnas_tabla
    time_tabla = pd.concat([time_tabla, temp_tabla], axis=0)

    temp_tabla = pd.DataFrame(se_list).transpose()
    temp_tabla.columns = columnas_tabla
    se_tabla = pd.concat([se_tabla, temp_tabla], axis=0)

    temp_tabla = pd.DataFrame(sp_list).transpose()
    temp_tabla.columns = columnas_tabla
    sp_tabla = pd.concat([sp_tabla, temp_tabla], axis=0)

    temp_tabla = pd.DataFrame(roc_list).transpose()
    temp_tabla.columns = columnas_tabla
    roc_tabla = pd.concat([roc_tabla, temp_tabla], axis=0)

    temp_tabla = pd.DataFrame(p_list).transpose()
    temp_tabla.columns = columnas_tabla
    p_tabla = pd.concat([p_tabla, temp_tabla], axis=0)

Paso 5. Mostramos la tablas de resultados, para analizarlas

Mostramos el tiempo de cómputo

In [ ]:
print('Tiempo de cómputo')
time_tabla

Mostramos la sensibilidad


In [ ]:
print('Sensibilidad')
se_tabla

Mostramos la especificidad

In [ ]:
print('Especificidad')
sp_tabla

Mostramos la precisión

In [ ]:
print('Precisión')
p_tabla

Mostramos el área bajo la curva roc

In [ ]:
print('ROC')
roc_tabla